#Installations


In [ ]:
!pip install -q langchain langchain-community langchain-google-genai chromadb llama-index pdf2image pytesseract unstructured tiktoken opencv-python-headless

!apt install -y poppler-utils tesseract-ocr tesseract-ocr-ben

#PDF to OCR & Extraction

In [ ]:
from pdf2image import convert_from_path
from pathlib import Path
from PIL import Image, ImageEnhance
import pytesseract
import cv2
import numpy as np
import os


pdf_path = "/content/HSC26-Bangla1st-Paper.pdf"
output_dir = Path("/content/pdf_images")
output_dir.mkdir(exist_ok=True)


# Convert PDF to Images (Pages 6–17)

print("Converting PDF pages 6–17 to images...")
images = convert_from_path(pdf_path, dpi=300, first_page=6, last_page=17)
image_paths = []


# Image Enhancement for OCR

def enhance_image(image_pil):
    # Convert to OpenCV BGR format
    image_bgr = np.array(image_pil)[:, :, ::-1]

    # Apply sharpening kernel
    kernel = np.array([[0, -1, 0],
                       [-1,  5, -1],
                       [0, -1, 0]])
    sharpened = cv2.filter2D(image_bgr, -1, kernel)

    # Convert back to PIL and increase contrast
    image_rgb = cv2.cvtColor(sharpened, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(image_rgb)
    contrast_enhancer = ImageEnhance.Contrast(pil_image)
    final_image = contrast_enhancer.enhance(1.5)

    return final_image

# Process and save each enhanced page
for i, img in enumerate(images):
    page_num = i + 6
    enhanced_img = enhance_image(img)
    img_path = output_dir / f"page_{page_num}_enhanced.png"
    enhanced_img.save(img_path)
    image_paths.append(str(img_path))
    print(f"Page {page_num} enhanced and saved.")


# Run OCR on Enhanced Images

pytesseract.pytesseract.tesseract_cmd = "/usr/bin/tesseract"
full_text = ""

print("\n Running OCR on enhanced images...")
for i, img_path in enumerate(image_paths):
    try:
        with Image.open(img_path) as img:
            text = pytesseract.image_to_string(img, lang="ben")
            full_text += f"\n--- Page {i + 6} ---\n{text.strip()}\n"

            # Save individual page text
            txt_path = img_path.replace(".png", ".txt")
            with open(txt_path, "w", encoding="utf-8") as f:
                f.write(text)

            print(f"OCR complete for page {i + 6}")

    except Exception as e:
        print(f"Error processing page {i + 6}: {e}")


# Save All Text to One File

final_txt_path = "/content/final_extracted_text.txt"
with open(final_txt_path, "w", encoding="utf-8") as f:
    f.write(full_text)

print("\n All OCR completed and saved to:", final_txt_path)


# GEMINI API

In [ ]:
from google.colab import userdata

api_key = userdata.get('GEMINI_API_KEY')

#Chunking, Embedding, and Storing in ChromaDB

In [ ]:
import shutil
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores.chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# --- Chunking ---
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200) #----------------- Need to experiment with chunk size and overlap ----------
chunks = text_splitter.split_text(full_text)
print(f"Total chunks created: {len(chunks)}")

# --- Embedding and Vector Store ---

embedding_model = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004", google_api_key=api_key)

print("Creating in-memory vector store...")

# By omitting 'persist_directory', the database is created in-memory.
vectorstore = Chroma.from_texts(
    texts=chunks,
    embedding=embedding_model
)

print("In-memory vector store created successfully.")

Total chunks created: 35
Creating in-memory vector store...
In-memory vector store created successfully.


#RAG Chain with Memory

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from operator import itemgetter

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser


# This dictionary will hold the history for each unique session.
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    """
    Retrieves the chat history for a given session.
    If no history exists for the session_id, a new one is created.
    """
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


# conversational prompt template
conversational_prompt = ChatPromptTemplate.from_messages([
    ("system",
     """
      তুমি একজন বিশেষজ্ঞ সহকারী, যার কাজ HSC বাংলা সাহিত্য নিয়ে প্রশ্নের উত্তর দেয়া।
      শুধু নিচের প্রাসঙ্গিক তথ্য (CONTEXT) দেখে প্রশ্নের উত্তর দাও।


      প্রাসঙ্গিক তথ্য (CONTEXT):
      {context}

      প্রশ্ন (QUESTION):
      {question}

      উত্তর (ANSWER):

      """),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}")
])

retriever = vectorstore.as_retriever(search_kwargs={"k": 15})
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite", temperature=0.2, google_api_key=api_key)

# RAG chain
rag_chain = (
    RunnablePassthrough.assign(
        context=itemgetter("question") | retriever
    )
    | conversational_prompt
    | llm
    | StrOutputParser()
)


# RAG chain with memory management

chain_with_memory = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="chat_history",
)


# Test 
session_id_1 = "user_hsc_rag_session_01"
config = {"configurable": {"session_id": session_id_1}}

# --- First Question ---
print("--- Asking First Question ---")
question1 = "অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?"
response1 = chain_with_memory.invoke({"question": question1}, config=config)
print(f"Q: {question1}")
print(f"A: {response1}\n")


# --- Follow-Up Question ---
print("--- Asking Follow-Up Question ---")
question1 = "শম্ভুনাথ কে ছিল?"
response1 = chain_with_memory.invoke({"question": question1}, config=config)
print(f"Q: {question1}")
print(f"A: {response1}\n")


#Evaluation

In [ ]:
!pip install -q ragas datasets

In [ ]:
# Evaluation Dataset

from datasets import Dataset

eval_data = {
    'question': [
        # q1
        "অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?",
        # q2
        "কাকে অনুপমের/লেখকের ভাগ্যদেবতা বলে উল্লেখ করা হয়েছে?",
        # q3
        "বিয়ের সময় কল্যাণীর প্রকৃত বয়স কত ছিল?"
    ],
    'ground_truth': [
        # gt1
        "শম্ভুনাথ",
        # gt2
        "মামা",
        # gt3
        "পনেরো"
    ],
    'contexts': [
        # exact context snippet for q1 as a list containing one string
        ["বলা বাহুল্য, বিবাহ-উপলক্ষে কন্যাপক্ষকেই কলিকাতা আসিতে হইল। কন্যার পিতা শম্ভুনাথ বাবু হরিশকে কত বিশ্বাস করেন তাহার প্রমাণ এই যে, বিবাহের তিন দিন পূর্বে তিনি আমাকে চক্ষে দেখেন এবং আশীর্বাদ করিয়া যান। বয়স তার চল্লিশের কিছু এপারে বা ওপারে। চুল কাঁচা, গোঁফে পাক ধরিতে আরম্ভ করিয়াছে মাত্র। সুপুরুষ বটে। ভিড়ের মধ্যে দেখিলে সকলের আগে তার উপরে চোখ পড়িবার মতো চেহারা।"],

        #  exact context snippet for q2 as a list containing one string
        ["কিন্তু মামা, যিনি পৃথিবীতে আমার ভাগ্যদেবতার প্রধান এজেন্ট, বিবাহ সন্বন্ধে তার একটা বিশেষ মত ছিল।"],

        #  exact context snippet for q3 as a list containing one string
        ["এসব ভালো কথা। কিন্তু, মেয়ের বয়স যে পনেরো, তাই শুনিয়া মামার মন ভার হইল।"]

    ]
}


eval_dataset = Dataset.from_dict(eval_data)


import pandas as pd
from ragas import evaluate

from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

results = []
# Loop to iterate through questions and get answers
# Using i for unique session IDs

for i, item in enumerate(eval_dataset):
    question = item['question']

    # Create a unique session ID for each question
    config = {"configurable": {"session_id": f"eval_session_{i}"}}

    answer = chain_with_memory.invoke({"question": question}, config=config)

    retrieved_contexts = [doc.page_content for doc in retriever.invoke(question)]

    results.append({
        "question": question,
        "answer": answer,
        "contexts": retrieved_contexts,
        "ground_truth": item['ground_truth']
    })

results_dataset = Dataset.from_list(results)


# Run the RAGAs Evaluation

evaluation_result = evaluate(
    dataset=results_dataset,
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
    llm=llm,
    embeddings=embedding_model
)


# Prnit the results

df_results = evaluation_result.to_pandas()
print("\n--- RAG Evaluation Results ---")
pd.set_option('display.max_colwidth', None)
display(df_results.style.background_gradient(cmap='viridis', subset=['context_precision', 'context_recall', 'faithfulness', 'answer_relevancy']))

Evaluating:   0%|          | 0/12 [00:00<?, ?it/s]

  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.0-flash-lite"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 30
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 47
}
].
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.0-flash-lite"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 30
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 47
}
].
  quota_metric: "generativelanguage.googleapis.com


--- RAG Evaluation Results ---


#API

In [ ]:
!pip install -q fastapi uvicorn pyngrok

from fastapi import FastAPI
from pydantic import BaseModel
from typing import Any
import uvicorn
from pyngrok import ngrok
import asyncio
import os

# SETUP NGROK 
NGROK_AUTH_TOKEN = "30Mp4YhMiIT6evuX74fObq8eISF_6dRn6Qz8sYvnBQBdMrUat"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# FASTAPI APPLICATION DEFINITION
app = FastAPI(
    title="HSC Bangla RAG API",
    description="An API for interacting with a conversational RAG system.",
    version="1.0.0",
)

class ChatRequest(BaseModel):
    session_id: str
    question: str

class ChatResponse(BaseModel):
    session_id: str
    answer: Any

@app.post("/chat", response_model=ChatResponse)
async def chat_endpoint(request: ChatRequest):
    """
    Endpoint to handle a chat interaction.
    """
    session_id = request.session_id
    question = request.question
    config = {"configurable": {"session_id": session_id}}

    answer = await chain_with_memory.ainvoke({"question": question}, config=config)

    return ChatResponse(session_id=session_id, answer=answer)

# RUN THE API WITH NGROK 

async def run_server():
    # Set up the public URL to your Colab instance
    public_url = ngrok.connect(8000)
    print(f"🚀 API is live! Access the interactive docs at: {public_url}/docs")

    # Configure and start the FastAPI server
    config = uvicorn.Config(app=app, port=8000, loop="asyncio")
    server = uvicorn.Server(config)
    await server.serve()

try:
    await run_server()
except KeyboardInterrupt:
    print("Shutting down ngrok tunnel and server.")
    ngrok.kill()


#END